In [1]:
# Graduation Prediction Model Training

## Overview
# This notebook trains a Logistic Regression model using XGBoost to predict whether students will graduate on-time or late based on their academic performance patterns.

## Key Features
# - Extracts course-level performance data from SQL Server database
# - Engineers 25+ predictive features including attempt patterns, failure rates, and score statistics
# - Trains calibrated XGBoost classifier with class imbalance handling
# - Saves trained model with metadata and feature importance analysis

## Dataset
# - **Source**: STUDENT_SCORE and STUDENTS tables in SQL Server
# - **Target Variable**: `on_time` (binary: 1 = on-time graduation, 0 = late graduation)
# - **Features**: Course attempts, failure patterns, first-attempt scores, resit rates, etc.


## 1. Setup and Imports

# System path configuration
import os
import sys

# Add project root to path
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.insert(0, project_root)

# Database connection
from src.db.core import get_db_connection

# Data processing and ML libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.calibration import CalibratedClassifierCV
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, classification_report
import joblib
import json
import logging
from datetime import datetime

# Configure logging
logging.basicConfig(filename='model_training.log', level=logging.INFO)

print("✓ All libraries imported successfully")
print(f"Working directory: {os.getcwd()}")


✓ All libraries imported successfully
Working directory: c:\Users\Justyn Lim\Desktop\Python\FYP\SoC-SMS\src\services


In [2]:
## 2. Data Extraction

# Extract features and labels from the database using a comprehensive SQL query that:
# - Calculates course attempt patterns for each student
# - Computes performance metrics (scores, pass rates, failure counts)
# - Determines on-time vs late graduation status
# - Engineers derived features like resit rates and first-attempt pass rates


def extract_features_and_labels():
    """Query database to get features and labels for graduated students"""
    
    conn = get_db_connection()

    query = """
    WITH student_entry_year AS (
        SELECT 
            MATRIC_NO,
            CASE 
                WHEN MAX(CASE WHEN ATTEMPT_1 = 'Exempted' THEN 1 ELSE 0 END) = 1 
                THEN 2
                ELSE 1
            END AS entry_year_level,
            CASE 
                WHEN MAX(CASE WHEN ATTEMPT_1 = 'Exempted' THEN 1 ELSE 0 END) = 1 
                THEN 2
                ELSE 3
            END AS expected_years
        FROM STUDENT_SCORE
        GROUP BY MATRIC_NO
    ),

    course_attempt_details AS (
        SELECT 
            MATRIC_NO,
            COURSE_CODE,
            ATTEMPT_1,
            ATTEMPT_2,
            ATTEMPT_3,
            
            CASE 
                WHEN ATTEMPT_1 NOT IN ('-', 'Exempted') 
                THEN 1 ELSE 0 
            END +
            CASE 
                WHEN ATTEMPT_2 != '-' 
                THEN 1 ELSE 0 
            END +
            CASE 
                WHEN ATTEMPT_3 NOT IN ('-', 'NULL')
                THEN 1 ELSE 0 
            END AS attempts_for_course,
            
            CASE 
                WHEN ATTEMPT_1 NOT IN ('Exempted', '-') 
                     AND ISNUMERIC(ATTEMPT_1) = 1 
                     AND CAST(ATTEMPT_1 AS FLOAT) < 40
                THEN 1 ELSE 0 
            END AS failed_first_attempt,
            
            CASE WHEN ATTEMPT_1 = 'Exempted' THEN 1 ELSE 0 END AS is_exempted,
            
            CASE 
                WHEN ATTEMPT_1 NOT IN ('Exempted', '-')
                     AND ISNUMERIC(ATTEMPT_1) = 1 
                THEN CAST(ATTEMPT_1 AS FLOAT)
                ELSE NULL
            END AS first_attempt_score,
            
            CASE 
                WHEN ATTEMPT_3 NOT IN ('-', 'NULL')
                     AND ISNUMERIC(ATTEMPT_3) = 1 
                THEN CAST(ATTEMPT_3 AS FLOAT)
                
                WHEN ATTEMPT_2 != '-'
                     AND ISNUMERIC(ATTEMPT_2) = 1 
                THEN CAST(ATTEMPT_2 AS FLOAT)
                
                WHEN ATTEMPT_1 NOT IN ('Exempted', '-')
                     AND ISNUMERIC(ATTEMPT_1) = 1 
                THEN CAST(ATTEMPT_1 AS FLOAT)
                
                ELSE NULL
            END AS final_score,
            
            CASE 
                WHEN (
                    (ATTEMPT_3 NOT IN ('-', 'NULL') AND ISNUMERIC(ATTEMPT_3) = 1 AND CAST(ATTEMPT_3 AS FLOAT) >= 40)
                    OR (ATTEMPT_2 != '-' AND ISNUMERIC(ATTEMPT_2) = 1 AND CAST(ATTEMPT_2 AS FLOAT) >= 40)
                    OR (ATTEMPT_1 NOT IN ('Exempted', '-') AND ISNUMERIC(ATTEMPT_1) = 1 AND CAST(ATTEMPT_1 AS FLOAT) >= 40)
                )
                THEN 1 ELSE 0 
            END AS eventually_passed
            
        FROM STUDENT_SCORE
    ),

    student_features_enhanced AS (
        SELECT 
            MATRIC_NO,
            
            COUNT(DISTINCT COURSE_CODE) as total_courses,
            SUM(is_exempted) as exempted_courses,
            COUNT(DISTINCT COURSE_CODE) - SUM(is_exempted) as actual_courses_taken,
            
            SUM(CASE WHEN attempts_for_course = 1 THEN 1 ELSE 0 END) as courses_passed_first_attempt,
            SUM(CASE WHEN attempts_for_course = 2 THEN 1 ELSE 0 END) as courses_with_2_attempts,
            SUM(CASE WHEN attempts_for_course = 3 THEN 1 ELSE 0 END) as courses_with_3_attempts,
            SUM(CASE WHEN attempts_for_course >= 2 THEN 1 ELSE 0 END) as total_courses_needing_resits,
            
            SUM(failed_first_attempt) as total_first_attempt_failures,
            SUM(CASE WHEN failed_first_attempt = 1 AND eventually_passed = 0 THEN 1 ELSE 0 END) as courses_never_passed,
            SUM(CASE WHEN failed_first_attempt = 1 AND eventually_passed = 1 THEN 1 ELSE 0 END) as courses_passed_after_failing,
            
            AVG(first_attempt_score) as avg_first_attempt_score,
            MIN(first_attempt_score) as lowest_first_attempt_score,
            STDEV(first_attempt_score) as first_attempt_score_std_dev,
            
            AVG(final_score) as avg_final_score,
            MIN(final_score) as lowest_final_score,
            MAX(final_score) as highest_final_score,
            
            SUM(CASE WHEN first_attempt_score >= 70 THEN 1 ELSE 0 END) as courses_with_distinction_first_attempt,
            SUM(CASE WHEN first_attempt_score >= 40 AND first_attempt_score < 50 THEN 1 ELSE 0 END) as courses_barely_passed_first_attempt,
            SUM(CASE WHEN final_score = 40 AND attempts_for_course > 1 THEN 1 ELSE 0 END) as courses_capped_at_40,
            SUM(CASE WHEN final_score < 40 THEN 1 ELSE 0 END) as courses_still_failing
            
        FROM course_attempt_details
        GROUP BY MATRIC_NO
    ),

    graduation_labels AS (
        SELECT 
            s.MATRIC_NO,
            s.COHORT,
            s.GRADUATED_ON,
            sey.entry_year_level,
            sey.expected_years,
            
            DATEPART(YEAR, s.COHORT) AS entry_year,
            DATEPART(MONTH, s.COHORT) AS entry_month,
            
            2000 + CAST(SUBSTRING(s.GRADUATED_ON, 2, 2) AS INT) AS grad_year,
            CAST(SUBSTRING(s.GRADUATED_ON, 5, 1) AS INT) AS grad_month,
            
            DATEPART(YEAR, DATEADD(YEAR, sey.expected_years, s.COHORT)) AS expected_grad_year,
            DATEPART(MONTH, s.COHORT) AS expected_grad_month,
            
            CASE 
                WHEN 2000 + CAST(SUBSTRING(s.GRADUATED_ON, 2, 2) AS INT) < DATEPART(YEAR, DATEADD(YEAR, sey.expected_years, s.COHORT)) THEN 1
                WHEN 2000 + CAST(SUBSTRING(s.GRADUATED_ON, 2, 2) AS INT) = DATEPART(YEAR, DATEADD(YEAR, sey.expected_years, s.COHORT)) 
                     AND CAST(SUBSTRING(s.GRADUATED_ON, 5, 1) AS INT) <= DATEPART(MONTH, s.COHORT) THEN 1
                ELSE 0
            END AS on_time
            
        FROM STUDENTS s
        INNER JOIN student_entry_year sey ON s.MATRIC_NO = sey.MATRIC_NO
        WHERE 
            s.STUDENT_STATUS = 'Graduate'
            AND s.GRADUATED_ON != '-'
            AND s.GRADUATED_ON IS NOT NULL
    )

    SELECT 
        gl.*,
        
        sfe.total_courses,
        sfe.exempted_courses,
        sfe.actual_courses_taken,
        sfe.courses_passed_first_attempt,
        sfe.courses_with_2_attempts,
        sfe.courses_with_3_attempts,
        sfe.total_courses_needing_resits,
        sfe.total_first_attempt_failures,
        sfe.courses_never_passed,
        sfe.courses_passed_after_failing,
        sfe.avg_first_attempt_score,
        sfe.lowest_first_attempt_score,
        sfe.first_attempt_score_std_dev,
        sfe.avg_final_score,
        sfe.lowest_final_score,
        sfe.highest_final_score,
        sfe.courses_with_distinction_first_attempt,
        sfe.courses_barely_passed_first_attempt,
        sfe.courses_capped_at_40,
        sfe.courses_still_failing,
        
        CAST(sfe.courses_passed_first_attempt AS FLOAT) / NULLIF(sfe.actual_courses_taken, 0) as first_attempt_pass_rate,
        CAST(sfe.total_courses_needing_resits AS FLOAT) / NULLIF(sfe.actual_courses_taken, 0) as resit_rate,
        CAST(sfe.total_first_attempt_failures AS FLOAT) / NULLIF(sfe.actual_courses_taken, 0) as first_attempt_failure_rate,
        CAST(sfe.courses_with_3_attempts AS FLOAT) / NULLIF(sfe.total_courses_needing_resits, 0) as third_attempt_rate,
        CAST(sfe.courses_capped_at_40 AS FLOAT) / NULLIF(sfe.total_courses_needing_resits, 0) as resit_success_rate
        
    FROM graduation_labels gl
    INNER JOIN student_features_enhanced sfe ON gl.MATRIC_NO = sfe.MATRIC_NO
    ORDER BY gl.MATRIC_NO
    """
    
    df = pd.read_sql(query, conn)
    conn.close()
    
    logging.info(f"Extracted {len(df)} training records from database")
    print(f"✓ Extracted {len(df)} training records")
    return df

# Execute data extraction
df = extract_features_and_labels()


✓ Extracted 0 training records


C:\Users\Justyn Lim\AppData\Local\Temp\ipykernel_35712\3551827980.py:199: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [3]:
## 3. Exploratory Data Analysis

# Examine the extracted dataset to understand class distribution and data quality.

print(f"Training data shape: {df.shape}")
print(f"\nColumn names:\n{df.columns.tolist()}")
print(f"\nOn-time graduation distribution:\n{df['on_time'].value_counts()}")
print(f"\nClass percentages:")
print(df['on_time'].value_counts(normalize=True) * 100)

# Display sample records
print(f"\nSample records:")
df.head()


Training data shape: (0, 37)

Column names:
['MATRIC_NO', 'COHORT', 'GRADUATED_ON', 'entry_year_level', 'expected_years', 'entry_year', 'entry_month', 'grad_year', 'grad_month', 'expected_grad_year', 'expected_grad_month', 'on_time', 'total_courses', 'exempted_courses', 'actual_courses_taken', 'courses_passed_first_attempt', 'courses_with_2_attempts', 'courses_with_3_attempts', 'total_courses_needing_resits', 'total_first_attempt_failures', 'courses_never_passed', 'courses_passed_after_failing', 'avg_first_attempt_score', 'lowest_first_attempt_score', 'first_attempt_score_std_dev', 'avg_final_score', 'lowest_final_score', 'highest_final_score', 'courses_with_distinction_first_attempt', 'courses_barely_passed_first_attempt', 'courses_capped_at_40', 'courses_still_failing', 'first_attempt_pass_rate', 'resit_rate', 'first_attempt_failure_rate', 'third_attempt_rate', 'resit_success_rate']

On-time graduation distribution:
Series([], Name: count, dtype: int64)

Class percentages:
Series([],

,MATRIC_NO,COHORT,GRADUATED_ON,entry_year_level,expected_years,entry_year,entry_month,grad_year,grad_month,expected_grad_year,...,highest_final_score,courses_with_distinction_first_attempt,courses_barely_passed_first_attempt,courses_capped_at_40,courses_still_failing,first_attempt_pass_rate,resit_rate,first_attempt_failure_rate,third_attempt_rate,resit_success_rate


In [4]:
# Check for missing values in key features
print("Missing values per column:")
df.isnull().sum()[df.isnull().sum() > 0]

# Basic statistics
df.describe()


Missing values per column:


,MATRIC_NO,COHORT,GRADUATED_ON,entry_year_level,expected_years,entry_year,entry_month,grad_year,grad_month,expected_grad_year,...,highest_final_score,courses_with_distinction_first_attempt,courses_barely_passed_first_attempt,courses_capped_at_40,courses_still_failing,first_attempt_pass_rate,resit_rate,first_attempt_failure_rate,third_attempt_rate,resit_success_rate
count,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
unique,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
## 4. Data Validation

# Check if we have sufficient data for training, especially for the minority class (late graduates).

if len(df) == 0:
    raise ValueError("ERROR: No training data found!")

on_time_count = df['on_time'].sum()
late_count = len(df) - on_time_count

print(f"Class distribution:")
print(f"  - On-time graduates: {on_time_count} ({on_time_count/len(df)*100:.1f}%)")
print(f"  - Late graduates: {late_count} ({late_count/len(df)*100:.1f}%)")

if late_count == 0:
    raise ValueError("⚠ WARNING: No late graduates found! Cannot train binary classifier.")

if late_count < 5 or on_time_count < 5:
    print(f"\n⚠ WARNING: Very limited data for one class!")
    print(f"Training will proceed without probability calibration due to small sample size.")
    use_calibration = False
else:
    use_calibration = True
    
print(f"\nCalibration: {'Enabled' if use_calibration else 'Disabled'}")


ValueError: ERROR: No training data found!

In [ ]:
## 5. Feature Engineering

# Define the feature set for model training. These features capture academic performance patterns that predict graduation timing.

### Feature Categories:
# 1. **Basic**: Entry year level, course counts
# 2. **Attempt Patterns**: Number of courses requiring 1, 2, or 3 attempts
# 3. **Failure Patterns**: First-attempt failures, never-passed courses
# 4. **Score Metrics**: Average, minimum, standard deviation of scores
# 5. **Derived Ratios**: Pass rates, resit rates, failure rates (key predictors)


# Define feature columns
feature_cols = [
    # Basic
    'entry_year_level',
    'total_courses',
    'actual_courses_taken',
    
    # Attempt patterns (KEY PREDICTORS)
    'courses_passed_first_attempt',
    'courses_with_2_attempts',
    'courses_with_3_attempts',
    'total_courses_needing_resits',
    
    # Failure patterns (STRONG PREDICTORS)
    'total_first_attempt_failures',
    'courses_never_passed',
    'courses_passed_after_failing',
    
    # First attempt performance (VERY STRONG PREDICTORS)
    'avg_first_attempt_score',
    'lowest_first_attempt_score',
    'first_attempt_score_std_dev',
    
    # Final performance
    'avg_final_score',
    'lowest_final_score',
    
    # Success indicators
    'courses_with_distinction_first_attempt',
    'courses_barely_passed_first_attempt',
    'courses_capped_at_40',
    
    # Derived ratios (CRITICAL PREDICTORS)
    'first_attempt_pass_rate',
    'resit_rate',
    'first_attempt_failure_rate',
    'third_attempt_rate',
    'resit_success_rate'
]

print(f"Total features: {len(feature_cols)}")
print("\nFeature list:")
for i, feat in enumerate(feature_cols, 1):
    print(f"{i:2d}. {feat}")


In [ ]:
# Prepare feature matrix and target variable
X = df[feature_cols].fillna(0)
y = df['on_time']

print(f"Feature matrix shape: {X.shape}")
print(f"Target variable shape: {y.shape}")
print(f"\nFeature data types:\n{X.dtypes.value_counts()}")


In [ ]:
## 6. Train-Test Split

# Split data into training and test sets. For very small datasets (< 20 samples), use all data for training.

# Determine if we should create a test set
if len(df) < 20 or late_count < 5:
    print(f"⚠ Using all {len(df)} samples for training (no test set due to small sample size)")
    X_train, X_test = X, None
    y_train, y_test = y, None
else:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42
    )
    print(f"Training set: {len(X_train)} samples")
    print(f"Test set: {len(X_test)} samples")

print(f"\nTraining set class distribution:")
print(y_train.value_counts())



In [ ]:
## 7. Model Configuration

# Configure XGBoost classifier with class imbalance handling using `scale_pos_weight`.

# Calculate class weights
pos = y_train.sum()
neg = (y_train == 0).sum()
scale_pos_weight = max(1.0, neg / max(1, pos))

print(f"Class balance in training set:")
print(f"  - On-time (positive): {pos}")
print(f"  - Late (negative): {neg}")
print(f"  - Scale weight: {scale_pos_weight:.2f}")

# Initialize XGBoost classifier
xgb = XGBClassifier(
    objective='binary:logistic',
    eval_metric='auc',
    tree_method='hist',
    random_state=42,
    n_estimators=200,
    learning_rate=0.1,
    max_depth=3,
    scale_pos_weight=scale_pos_weight,
    n_jobs=-1
)

print("\n✓ XGBoost classifier configured")
print(f"Parameters: {xgb.get_params()}")


In [ ]:
## 8. Model Training

# Train the XGBoost model and optionally calibrate probabilities using CalibratedClassifierCV.


# Train base XGBoost model
print("Training XGBoost model...")
xgb.fit(X_train, y_train)
print("✓ Base model training complete")

# Calibrate probabilities if sufficient data
if use_calibration and neg >= 3 and pos >= 3:
    print("\nCalibrating probabilities...")
    try:
        cv_folds = 2 if min(neg, pos) < 5 else 3
        xgb_cal = CalibratedClassifierCV(xgb, method='sigmoid', cv=cv_folds)
        xgb_cal.fit(X_train, y_train)
        final_model = xgb_cal
        print(f"✓ Calibration successful (cv={cv_folds})")
    except Exception as e:
        print(f"⚠ Calibration failed: {e}")
        print("Using uncalibrated model instead.")
        final_model = xgb
else:
    print("\n⚠ Skipping calibration due to insufficient data")
    final_model = xgb

print("\n✓ Final model ready")


In [ ]:
## 9. Model Evaluation

# Evaluate model performance using AUC-ROC and classification report.


# Evaluate on test set if available, otherwise on training set
if X_test is not None and y_test is not None:
    print("="*60)
    print("Model Performance (Test Set)")
    print("="*60)
    
    y_pred_proba = final_model.predict_proba(X_test)[:, 1]
    y_pred = (y_pred_proba >= 0.5).astype(int)
    
    auc = roc_auc_score(y_test, y_pred_proba)
    
    print(f"\nAUC-ROC: {auc:.4f}")
    print(f"\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=['Late', 'On-time'], zero_division=0))
else:
    print("="*60)
    print("Model Performance (Training Set - no test set available)")
    print("="*60)
    
    y_pred_proba = final_model.predict_proba(X_train)[:, 1]
    y_pred = (y_pred_proba >= 0.5).astype(int)
    
    auc = roc_auc_score(y_train, y_pred_proba)
    
    print(f"\nAUC-ROC: {auc:.4f} (Warning: May be optimistic)")
    print(f"\nClassification Report:")
    print(classification_report(y_train, y_pred, target_names=['Late', 'On-time'], zero_division=0))

# Store AUC for later use
model_auc = auc


In [ ]:
## 10. Feature Importance Analysis

# Analyze which features contribute most to predictions.


# Extract feature importance
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': xgb.feature_importances_
}).sort_values('importance', ascending=False)

print("Top 10 Most Important Features:")
print(feature_importance.head(10).to_string(index=False))

print(f"\nTop 5 features contribute {feature_importance.head(5)['importance'].sum():.2%} of total importance")



In [ ]:
# Visualize feature importance (top 15)
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
top_features = feature_importance.head(15)
plt.barh(range(len(top_features)), top_features['importance'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Importance')
plt.title('Top 15 Most Important Features')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


In [ ]:
## 11. Save Model and Metadata

# Save the trained model, feature columns, metadata, and feature importance for deployment.


# Create models directory if it doesn't exist
os.makedirs('models', exist_ok=True)

# Generate timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
model_path = f'models/graduation_model_{timestamp}.joblib'

# Save models
joblib.dump(final_model, model_path)
joblib.dump(final_model, 'models/graduation_model_latest.joblib')
joblib.dump(feature_cols, 'models/feature_columns.joblib')

print(f"✓ Model saved to {model_path}")
print(f"✓ Latest model: models/graduation_model_latest.joblib")
print(f"✓ Feature columns: models/feature_columns.joblib")



In [ ]:
# Save metadata
metadata = {
    'timestamp': timestamp,
    'total_samples': len(df),
    'on_time_count': int(on_time_count),
    'late_count': int(late_count),
    'auc': float(model_auc),
    'calibrated': use_calibration and neg >= 3 and pos >= 3,
    'features': feature_cols,
    'feature_count': len(feature_cols)
}

with open('models/model_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("✓ Metadata saved to models/model_metadata.json")

# Display metadata
print("\nModel Metadata:")
for key, value in metadata.items():
    if key != 'features':  # Skip features list for readability
        print(f"  {key}: {value}")


In [ ]:
# Save feature importance
feature_importance_path = f'models/feature_importance_{timestamp}.csv'
feature_importance.to_csv(feature_importance_path, index=False)

print(f"✓ Feature importance saved to {feature_importance_path}")


In [ ]:
## 12. Training Summary

# Final summary of model training results and next steps.
print("="*60)
print("TRAINING COMPLETE!")
print("="*60)
print(f"\n✓ Model AUC: {model_auc:.4f}")
print(f"✓ Total samples: {len(df)}")
print(f"✓ Features used: {len(feature_cols)}")
print(f"✓ Calibration: {'Yes' if metadata['calibrated'] else 'No'}")
print(f"\n✓ Model saved to: {model_path}")
print(f"\nNOTE: Model trained with enhanced course-level features.")
print("Performance will improve as more graduates are added to the system.")
print("\nNext steps:")
print("  1. Review feature importance above")
print("  2. Test model predictions on new students")
print("  3. Integrate model into Flask API")
print("  4. Monitor performance and retrain periodically")
